# GraphRAG Query

## Prerequisites
Install 3rd party packages, not part of the Python Standard Library, to run the notebook

In [ ]:
! pip install --upgrade pip
! pip install python-magic
! pip install devtools  
! pip install requests 
! pip install tqdm

In [ ]:
import getpass
import json
import time
import requests
import magic
from pathlib import Path
from tqdm import tqdm
from devtools import pprint

headers = {}
index_name = "bike"
endpoint = "https://arg-syd-aiapp1day-ca.ashycoast-2c2322dd.eastus.azurecontainerapps.io"

## Query

Once an indexing job is complete, the knowledge graph is ready to query. Two types of queries (global and local) are currently supported. We encourage you to try both and experience the difference in responses. Note that query response time is also correlated to the TPM quota of the Azure OpenAI model you are using.

In [ ]:
# a helper function to parse out the result from a query response
def parse_query_response(
    response: requests.Response, return_context_data: bool = False
) -> requests.Response | dict[list[dict]]:
    """
    Print response['result'] value and return context data.
    """
    if response.ok:
        print(json.loads(response.text)["result"])
        if return_context_data:
            return json.loads(response.text)["context_data"]
        return response
    else:
        print(response.reason)
        print(response.content)
        return response


### Global Query 

Global queries are resource-intensive, but provide good responses to questions that require an understanding of the dataset as a whole.

In [ ]:
%%time

def global_search(
    index_name: str | list[str], query: str, community_level: int
) -> requests.Response:
    """Run a global query over the knowledge graph(s) associated with one or more indexes"""
    url = endpoint + "/query/global"
    # optional parameter: community level to query the graph at (default for global query = 1)
    request = {
        "index_name": index_name,
        "query": query,
        "community_level": community_level,
    }
    return requests.post(url, json=request, headers=headers)

# perform a global query
global_response = global_search(
    index_name=index_name,
    query="whats the top 2 bikes?",
    community_level=1,
)
global_response_data = parse_query_response(global_response, return_context_data=True)
global_response_data

### Local Query

Local search queries are best suited for narrow-focused questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?)

In [ ]:
%%time

def local_search(
    index_name: str | list[str], query: str, community_level: int
) -> requests.Response:
    """Run a local query over the knowledge graph(s) associated with one or more indexes"""
    url = endpoint + "/query/local"
    # optional parameter: community level to query the graph at (default for local query = 2)
    request = {
        "index_name": index_name,
        "query": query,
        "community_level": community_level,
    }
    return requests.post(url, json=request, headers=headers)


# perform a local query
local_response = local_search(
    index_name=index_name,
    query="tell me about TrailMax",
    community_level=2,
)
local_response_data = parse_query_response(local_response, return_context_data=True)
local_response_data